In [5]:
# !pip install selenium

In [6]:
# !pip install --upgrade pip

In [7]:
# !pip install webdriver_manager

In [8]:
!pip3 install bs4

In [10]:
# Import all required libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import requests
import re

In [151]:
# Create an instance of chrome browser using the chrome driver
service = Service(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service)

# Interact with ebay website
driver.get('https://www.amazon.in/')
time.sleep(2)

In [ ]:
#Navigate to Televisions page
element_elc = driver.find_element(By.XPATH, "//a[text()=' Electronics ']")
element_elc.click()
time.sleep(3)

element_elc = driver.find_element(By.XPATH, "//span[text()='Home Theater, TV & Video']")
element_elc.click()
time.sleep(3)

element_tel = driver.find_element(By.XPATH, "//span[text()='Televisions']")
element_tel.click()
time.sleep(3)


In [ ]:
#apply filter to select top brands
element_tv_size = driver.find_element(By.XPATH, "//span[@class='a-size-base a-color-base' and text()='Top Brands']")
element_tv_size.click()
time.sleep(3)


In [54]:
# load page data and return prodtuct details
def get_product_details():
    soup = BeautifulSoup(driver.page_source,'html.parser')
    soup_list = soup.find_all(class_ = "s-main-slot s-result-list s-search-results sg-row")
    
    # Create a BeautifulSoup object from the returned resutlset
    soup = BeautifulSoup(str(soup_list), 'html.parser')
    # Find all television items
    return soup.find_all('div', class_='s-result-item')

In [156]:
#Store the results obtained into a Dataframe and save it as CSV file. Two coloumns, product_name and product_price.
# Title column : Product Name, Price
try:
    products = []
    prices = []

    items = get_product_details()
    # Check if there is a next page and collect all product details into items list
    xpath_nextpage = "//a[text() = 'Next']"
    while(len(driver.find_elements(By.XPATH, xpath_nextpage)) > 0):
        element_nextpage = driver.find_elements(By.XPATH, xpath_nextpage)
        driver.execute_script("arguments[0].scrollIntoView();",element_nextpage[0])
        element_nextpage[0].click()
        time.sleep(3)
        items.extend(get_product_details())
        
    # print(items)
    # Iterate over the product list and extract the name, and price
    for item in items:
        name = item.find('span', class_='a-size-base-plus a-color-base a-text-normal')
        price = item.find('span', class_='a-price-whole')
        if(name != None and price != None):
            name = name.text.strip()
            price = 'INR ' + price.text.strip()
            products.append(name)
            prices.append(price)
            
    data = {'Product Name':products, 'Price':prices}
    df = pd.DataFrame(data)
    #df.head()
    # Save DataFrame to a CSV file
    df.to_csv('amazon_television_data.csv', index=False)
    driver.close()
    
except Exception as e:
    print('An error occurred during scraping:', str(e))

In [157]:
#Clean up the raw data
print(df.__len__())
df = df.drop_duplicates()
print(df.__len__())

179
179


In [158]:
df.head()

,Product Name,Price
0,Redmi 80 cm (32 inches) F Series HD Ready Smar...,"INR 12,999"
1,MI 80 cm (32 inches) A Series HD Ready Smart G...,"INR 12,499"
2,VW 80 cm (32 inches) Frameless Series HD Ready...,"INR 7,499"
3,VW 80 cm (32 inches) Frameless Series HD Ready...,"INR 6,799"
4,Redmi 108 cm (43 inches) 4K Ultra HD Android S...,"INR 22,999"


In [159]:
# Define the regular expression pattern for the brands
pattern = r'(LG|Samsung|Sony|TCL|Hisense|Panasonic|Sharp|Philips|Haier|Toshiba|MI|Redmi|VW|iFFALCON|Acer|OnePlus|Vu|Xiaomi|Blaupunkt|Kodak|Croma|Westinghouse|Onida|Dyanora|Sansui)'

# Extract the brand from the 'Product Name' column and assign it to the 'Brand' column
df['Brand'] = df['Product Name'].str.extract(pattern, flags=re.IGNORECASE, expand=False)

# Convert the 'Brand' column to uppercase
df['Brand'] = df['Brand'].str.upper()

In [161]:
# df.head()

In [218]:
# Convert the column to string type
df['Product Name'] = df['Product Name'].astype(str)

# Define the regular expression pattern for the size
pattern = r"(\d+)\s*(inch|inches|Inch)"
matches = df['Product Name'].apply(lambda x: re.findall(pattern, x))

# Extract the matches and assign to the 'Size' column
df['Size'] = [', '.join(match[0]) if match else None for match in matches]
df['Size'] = df['Size'].replace(', inch|, Inch', '', regex=True)
df.drop(df[df['Size'].isnull()].index, inplace=True)
 
# Sorting by column 'Size'
df.sort_values(by=['Size'], inplace=True)


In [176]:
# df.head()

In [219]:
# Define the regular expression pattern for the display
pattern = r'(LED|QLED|OLED)'

# Extract the brand from the 'Product Name' column and assign it to the 'Brand' column
df['Display'] = df['Product Name'].str.extract(pattern, flags=re.IGNORECASE, expand=False)

# Convert the 'Brand' column to uppercase
df['Display'] = df['Display'].str.upper()
df['Display'].fillna('LED', inplace=True)

In [167]:
# df.head()

In [220]:
df.to_csv('amazon_television_data.csv', index=False)

In [221]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177 entries, 54 to 159
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  177 non-null    object
 1   Price         177 non-null    object
 2   Brand         177 non-null    object
 3   Size          177 non-null    object
 4   Display       177 non-null    object
dtypes: object(5)
memory usage: 8.3+ KB


In [222]:
gk = df.groupby('Size')

In [225]:
gk.first()

,Product Name,Price,Brand,Display
Size,,,,
24,VW 60 cm (24 inches) Premium Series HD Ready L...,"INR 5,699",VW,LED
32,TCL 80.04 cm (32 inches) Bezel-Less S Series H...,"INR 12,490",TCL,LED
40,VW 101 cm (40 inches) Linux Series Frameless H...,"INR 12,499",VW,LED
42,Kodak 106 cm (42 inches) Full HD Certified And...,"INR 15,999",KODAK,LED
43,Panasonic 108 cm (43 inches) 4K Ultra HD Smart...,"INR 31,990",PANASONIC,LED
48,LG 121 cm (48 inches) 4K Ultra HD Smart OLED T...,"INR 75,990",LG,OLED
49,Samsung 123 cm (49 Inches) 4K UHD LED TV UE49M...,"INR 1,12,369",SAMSUNG,LED
50,Samsung 125 cm (50 inches) 4K Ultra HD Smart N...,"INR 89,990",SAMSUNG,QLED
55,Samsung 139.7 cm (55 inches) 4K Ultra HD Smart...,"INR 52,990",SAMSUNG,LED
